In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
pd.options.display.max_rows = 10
pd.options.display.max_columns = 500

In [2]:
DIVERSITY_UTILITY = 0.25
FRAC_ADMIT = 0.5


###
## Load dataframe where each row is a single applicant
###

df = pd.read_csv('./df_test.csv')
df['key'] = df['R'].astype(str) + "_"  + df['T'].astype(str)

###
## Estimate E[Y(1)|T, A]
###

df_train = pd.read_csv('./df_train.csv')
df_stratum_utility = df_train[['R','T','B_p']].groupby(['R','T']).mean().reset_index()
df_stratum_utility['stratum_utility'] = (df_stratum_utility['B_p'] + DIVERSITY_UTILITY * df_stratum_utility['R']).round(2)
df_stratum_utility['key'] = df_stratum_utility['R'].astype(str) + "_" + df_stratum_utility['T'].astype(str)

df = df.merge(df_stratum_utility[['stratum_utility','key']], on='key')
df['ml_outcomes'] = df['stratum_utility']

In [3]:
FRAC_ADMIT = 0.5#df[['A']].sum()/len(df)


In [4]:
FRAC_ADMIT

0.5

In [5]:
len(df)

100000

In [6]:
#df['ml_outcomes'] = df['R']

In [7]:
##
# R = race, T = test score, ml_outcomes = expected utility from admitting, 
# ml_outcomes_{} counterfactual utility given race 
##

df[['R','T','ml_outcomes','T_black','T_white','B_p']].sort_values(by='ml_outcomes')



,R,T,ml_outcomes,T_black,T_white,B_p
99985,0,14,0.03,13,14,0
99986,0,12,0.05,10,12,0
99935,0,19,0.06,15,19,0
99928,0,19,0.06,18,19,0
99929,0,19,0.06,14,19,0
...,...,...,...,...,...,...
99977,1,95,1.20,95,109,1
99994,1,94,1.23,94,108,1
99914,1,101,1.25,101,116,0
99996,1,104,1.25,104,121,1


In [8]:
#df['ml_outcomes'] = df['ml_outcomes'] + 1*df['R']

In [9]:
###
## Get total utility from admitting people in a stratum 
###

df_ = df[['R','T','B_p','ml_outcomes']].groupby(['R','T','B_p']).sum().reset_index()

In [10]:
###
## Get number of people in each stratum
###

df_count = df[['R','T','ml_outcomes','B_p']].groupby(['R','T','B_p']).count().reset_index()
df_count.columns = ['R','T','B_p','Count']
df_count['N'] = df_count['Count']

In [11]:
###
## Merge summary tables to get one table with Race, Test Score, SUM(Utility), COUNT(applicants) per stratum
###

dff = df_.merge(df_count[['N']],left_index=True,right_index=True).sort_values(by='ml_outcomes',ascending=False).reset_index().sort_values(by='index').reset_index()

In [12]:
# final info table
dff.sort_values(by='ml_outcomes')

,level_0,index,R,T,B_p,ml_outcomes,N
1,341,1,0,14,0,0.03,1
0,340,0,0,12,0,0.05,1
4,339,4,0,16,0,0.07,1
5,338,5,0,16,1,0.07,1
2,337,2,0,15,0,0.09,1
...,...,...,...,...,...,...,...
88,4,88,0,59,1,663.52,1144
98,3,98,0,64,1,667.32,996
100,2,100,0,65,1,669.30,970
90,1,90,0,60,1,684.60,1141


### Setup optimization problem 

In [13]:
from ortools.linear_solver import pywraplp


In [14]:
solver = pywraplp.Solver.CreateSolver('GLOP')


In [15]:
dff

,level_0,index,R,T,B_p,ml_outcomes,N
0,340,0,0,12,0,0.05,1
1,341,1,0,14,0,0.03,1
2,337,2,0,15,0,0.09,1
3,336,3,0,15,1,0.09,1
4,339,4,0,16,0,0.07,1
...,...,...,...,...,...,...,...
337,297,337,1,94,1,1.23,1
338,298,338,1,95,1,1.20,1
339,294,339,1,97,1,1.25,1
340,295,340,1,101,0,1.25,1


In [16]:
applicant_stratum = []
vars_cache = {}

# Objective: Maximize the expected utility of the admitted students
objective = solver.Objective()

# For each stratum
for ix, row in dff.iterrows():
    # probability of admission
    numvar = solver.NumVar(0.0, 1.0, str(ix))
    
    # store variable by index, and also by stratum R, T
    applicant_stratum.append(numvar)
    vars_cache[(row['R'],row['T'],row['B_p'])] = numvar
    
    # Benefit of admitting people is total utility in that stratum
    objective.SetCoefficient(applicant_stratum[ix], float(row['ml_outcomes']))
objective.SetMaximization()


In [17]:
# Currently we have no constraints 
solver.NumConstraints()

0

In [18]:
# Constraint: At most K applicants
K = int(len(df)*FRAC_ADMIT)
print(K)
admit_quota = solver.Constraint(0, K)

# Total applicants cannot exceed K 
for ix, row in dff.iterrows():
    admit_quota.SetCoefficient(applicant_stratum[ix], float(row['N']))

50000


In [19]:
# Now we have one constraint
solver.NumConstraints()

1

## Add Equalized Odds Constraints

In [20]:
## Make sure that you have to add all people in B_p stratum or none
## i.e. you can't add only people who pass boards and reject those who fail boards from same T, R stratum
didntexist, exists = 0, 0 

for ix, row in dff.iterrows():
    constrain_bp = solver.Constraint(0.0, 0.0)
    
    var1 = vars_cache[(row['R'],row['T'],row['B_p'])]
    key2 = (row['R'],row['T'], 1-row['B_p'])
    
    if key2 not in vars_cache:
        didntexist+=1
        continue
        
    var2 = vars_cache[key2]
    
    constrain_bp.SetCoefficient(var1, -1.0)
    constrain_bp.SetCoefficient(var2, 1.0)
    exists+=1

didntexist, exists

(44, 298)

In [21]:
white_pass_boards = []
white_fail_boards = []
black_pass_boards = []
black_fail_boards = []

for key in vars_cache:
    r, t, b_p = key
    if b_p == 1 and r==0:
        white_pass_boards.append(key)
    elif b_p == 0 and r==0:
        white_fail_boards.append(key)
    elif b_p == 1 and r==1:
        black_pass_boards.append(key)
    elif b_p == 0 and r==1:
        black_fail_boards.append(key)

len(white_pass_boards),len(white_fail_boards),len(black_pass_boards),len(black_fail_boards)

(99, 86, 80, 77)

In [22]:
NUM_TOTALS = {}
df_totals = dff[['N','R','B_p']].groupby(['R','B_p']).sum().reset_index()
for ix, row in df_totals.iterrows():
    NUM_TOTALS[(row['R'],row['B_p'])] = row['N']
    
N_IN_STRATAS = {}
for ix, row in dff.iterrows():
    N_IN_STRATAS[(row['R'],row['T'],row['B_p'])] = row['N']

In [23]:
# Now we have one constraint
solver.NumConstraints()

343

In [24]:
#Of those who pass the boards exams
#Frac white admitted and frac black admitted should be the same

constrain_pass_boards = solver.Constraint(0.0, 0.0)

for key in white_pass_boards:
    r, t, b_p = key
    N_IN_STRATUM = N_IN_STRATAS[(r,t,b_p)]
    N_TOTAL = NUM_TOTALS[(r,b_p)]
    
    constrain_pass_boards.SetCoefficient(vars_cache[key], float(N_IN_STRATUM) / float(N_TOTAL))

for key in black_pass_boards:
    r, t, b_p = key
    N_IN_STRATUM = N_IN_STRATAS[(r,t,b_p)]
    N_TOTAL = NUM_TOTALS[(r,b_p)]
    
    constrain_pass_boards.SetCoefficient(vars_cache[key], -1.0 * (float(N_IN_STRATUM) / float(N_TOTAL)))


In [25]:
#Of those who fail the boards exams
#Frac white admitted and frac black admitted should be the same

constrain_fail_boards = solver.Constraint(0.0, 0.0)

for key in white_fail_boards:
    r, t, b_p = key
    N_IN_STRATUM = N_IN_STRATAS[(r,t,b_p)]
    N_TOTAL = NUM_TOTALS[(r,b_p)]
    
    constrain_fail_boards.SetCoefficient(vars_cache[key], float(N_IN_STRATUM) / float(N_TOTAL))

for key in black_fail_boards:
    r, t, b_p = key
    N_IN_STRATUM = N_IN_STRATAS[(r,t,b_p)]
    N_TOTAL = NUM_TOTALS[(r,b_p)]
    
    constrain_fail_boards.SetCoefficient(vars_cache[key], -1.0 * (float(N_IN_STRATUM) / float(N_TOTAL)))


## Solve linear program

In [26]:
solver.ABNORMAL

4

In [27]:
status = solver.Solve()


In [28]:
status

0

In [29]:
solver.OPTIMAL

0

In [30]:
row = []
admit = []

for i in applicant_stratum:
    row.append(int(str(i)))
    admit.append(i.solution_value())

df_decisions = pd.DataFrame({'row_id':row,'decision':admit})

In [31]:
df_decisions

,row_id,decision
0,0,0.0
1,1,0.0
2,2,0.0
3,3,0.0
4,4,0.0
...,...,...
337,337,1.0
338,338,1.0
339,339,1.0
340,340,1.0


In [32]:
dff.merge(df_decisions,left_index=True,right_index=True).sort_values(by='ml_outcomes',ascending=False)

,level_0,index,R,T,B_p,ml_outcomes,N,row_id,decision
96,0,96,0,63,1,704.88,1068,96,1.000000
90,1,90,0,60,1,684.60,1141,90,0.743085
100,2,100,0,65,1,669.30,970,100,1.000000
98,3,98,0,64,1,667.32,996,98,1.000000
88,4,88,0,59,1,663.52,1144,88,1.000000
...,...,...,...,...,...,...,...,...,...
2,337,2,0,15,0,0.09,1,2,0.000000
5,338,5,0,16,1,0.07,1,5,0.000000
4,339,4,0,16,0,0.07,1,4,0.000000
0,340,0,0,12,0,0.05,1,0,0.000000


In [33]:
xxx_ = dff.merge(df_decisions,left_index=True,right_index=True).sort_values(by='ml_outcomes',ascending=False)
xxx_.sort_values(by='T')

,level_0,index,R,T,B_p,ml_outcomes,N,row_id,decision
185,331,185,1,5,0,0.25,1,185,0.0
186,330,186,1,8,0,0.25,1,186,0.0
187,321,187,1,10,0,0.54,2,187,0.0
0,340,0,0,12,0,0.05,1,0,0.0
188,329,188,1,13,0,0.27,1,188,0.0
...,...,...,...,...,...,...,...,...,...
180,278,180,0,112,1,3.00,3,180,1.0
181,288,181,0,113,1,2.00,2,181,1.0
182,305,182,0,114,1,1.00,1,182,1.0
183,306,183,0,116,1,0.99,1,183,1.0


In [34]:
xxx = dff.merge(df_decisions,left_index=True,right_index=True).sort_values(by='ml_outcomes',ascending=False)
xxx

,level_0,index,R,T,B_p,ml_outcomes,N,row_id,decision
96,0,96,0,63,1,704.88,1068,96,1.000000
90,1,90,0,60,1,684.60,1141,90,0.743085
100,2,100,0,65,1,669.30,970,100,1.000000
98,3,98,0,64,1,667.32,996,98,1.000000
88,4,88,0,59,1,663.52,1144,88,1.000000
...,...,...,...,...,...,...,...,...,...
2,337,2,0,15,0,0.09,1,2,0.000000
5,338,5,0,16,1,0.07,1,5,0.000000
4,339,4,0,16,0,0.07,1,4,0.000000
0,340,0,0,12,0,0.05,1,0,0.000000


In [35]:
xxx.to_csv('./decision.csv')

In [36]:
solution = pd.read_csv('./decision.csv')

In [37]:
xxx

,level_0,index,R,T,B_p,ml_outcomes,N,row_id,decision
96,0,96,0,63,1,704.88,1068,96,1.000000
90,1,90,0,60,1,684.60,1141,90,0.743085
100,2,100,0,65,1,669.30,970,100,1.000000
98,3,98,0,64,1,667.32,996,98,1.000000
88,4,88,0,59,1,663.52,1144,88,1.000000
...,...,...,...,...,...,...,...,...,...
2,337,2,0,15,0,0.09,1,2,0.000000
5,338,5,0,16,1,0.07,1,5,0.000000
4,339,4,0,16,0,0.07,1,4,0.000000
0,340,0,0,12,0,0.05,1,0,0.000000


In [38]:
df['key'] = df['R'].astype(str)+'_'+df['T'].astype(str)+'_'+df['B_p'].astype(str)
xxx['key'] = xxx['R'].astype(str)+'_'+xxx['T'].astype(str)+'_'+xxx['B_p'].astype(str)

In [39]:
xxx

,level_0,index,R,T,B_p,ml_outcomes,N,row_id,decision,key
96,0,96,0,63,1,704.88,1068,96,1.000000,0_63_1
90,1,90,0,60,1,684.60,1141,90,0.743085,0_60_1
100,2,100,0,65,1,669.30,970,100,1.000000,0_65_1
98,3,98,0,64,1,667.32,996,98,1.000000,0_64_1
88,4,88,0,59,1,663.52,1144,88,1.000000,0_59_1
...,...,...,...,...,...,...,...,...,...,...
2,337,2,0,15,0,0.09,1,2,0.000000,0_15_0
5,338,5,0,16,1,0.07,1,5,0.000000,0_16_1
4,339,4,0,16,0,0.07,1,4,0.000000,0_16_0
0,340,0,0,12,0,0.05,1,0,0.000000,0_12_0


In [40]:
admit_decisions = df.merge(xxx[['key','decision']],how='left',on='key')
admit_decisions['decision_random'] = pd.Series([random.random() for x in range(0,len(admit_decisions))]) < FRAC_ADMIT

FRAC_BLACK_POLICY = (admit_decisions['R'] * admit_decisions['decision']).sum()/admit_decisions['decision'].sum()
SUM_BP_POLICY = ( admit_decisions['B_p'] * admit_decisions['decision']).sum()

FRAC_RANDOM_POLICY = (admit_decisions['R'] * admit_decisions['decision_random']).sum()/admit_decisions['decision_random'].sum()
SUM_BP_RAND_POLICY = ( admit_decisions['B_p'] * admit_decisions['decision_random']).sum()

In [41]:
file = open('./lp_results.csv','a')
file.write('{}\t{}\t{}\n'.format('Counterfactual Equalized Odds',str(FRAC_BLACK_POLICY),str(SUM_BP_POLICY)))
file.close()

In [42]:
xxx[xxx['R']==0].sort_values(by='decision')

,level_0,index,R,T,B_p,ml_outcomes,N,row_id,decision,key
37,167,37,0,34,0,70.38,414,37,0.0,0_34_0
38,227,38,0,34,1,14.45,85,38,0.0,0_34_1
25,223,25,0,28,0,15.96,133,25,0.0,0_28_0
40,218,40,0,35,1,18.54,103,40,0.0,0_35_1
27,217,27,0,29,0,19.89,153,27,0.0,0_29_0
...,...,...,...,...,...,...,...,...,...,...
152,172,152,0,91,1,64.86,69,152,1.0,0_91_1
95,57,95,0,63,0,361.02,547,95,1.0,0_63_0
123,169,123,0,77,0,68.85,81,123,1.0,0_77_0
148,162,148,0,89,1,77.08,82,148,1.0,0_89_1


In [43]:
xxx[xxx['R']==1].sort_values(by='decision')

,level_0,index,R,T,B_p,ml_outcomes,N,row_id,decision,key
247,26,247,1,45,0,502.84,967,247,0.0,1_45_0
222,216,222,1,32,1,22.20,60,222,0.0,1_32_1
201,211,201,1,22,0,22.94,74,201,0.0,1_22_0
205,210,205,1,24,0,23.04,72,205,0.0,1_24_0
226,203,226,1,34,1,28.50,75,226,0.0,1_34_1
...,...,...,...,...,...,...,...,...,...,...
262,92,262,1,52,1,232.96,364,262,1.0,1_52_1
310,180,310,1,76,1,50.88,48,310,1.0,1_76_1
306,179,306,1,74,1,53.55,51,306,1.0,1_74_1
276,86,276,1,59,1,251.22,318,276,1.0,1_59_1


In [44]:
(xxx['N']*xxx['decision']).sum()/len(df)

0.49999999999999983

In [45]:
admit_decisions[admit_decisions['B_p']==1][['decision','R']].groupby(['R']).mean()

,decision
R,
0,0.678956
1,0.678956
